In [1]:
import pandas as pd
import numpy as np

# テストデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_clipped_outliers_transaction_date.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# データ型の確認
print("Data types of each column:\n", test_data.dtypes)

# 数値型の列を取得
numerical_cols = test_data.select_dtypes(include=['int64', 'float64']).columns
print("Numerical columns:\n", numerical_cols)

Data types of each column:
 ID              int64
種類             object
価格情報区分         object
地域            float64
市区町村コード         int64
都道府県名          object
市区町村名          object
地区名            object
最寄駅：名称         object
最寄駅：距離（分）     float64
間取り            object
面積（㎡）         float64
土地の形状         float64
間口            float64
延床面積（㎡）       float64
建築年             int64
建物の構造          object
用途             object
今後の利用目的        object
前面道路：方位       float64
前面道路：種類       float64
前面道路：幅員（ｍ）    float64
都市計画           object
建ぺい率（％）       float64
容積率（％）        float64
取引時点            int64
改装             object
取引の事情等         object
1953年以前          bool
dtype: object
Numerical columns:
 Index(['ID', '地域', '市区町村コード', '最寄駅：距離（分）', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）',
       '建築年', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '建ぺい率（％）', '容積率（％）', '取引時点'],
      dtype='object')


## 新しい特徴量を作成する

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

# テストデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_clipped_outliers_transaction_date.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# 取引時点から年を抽出
test_data['取引年'] = test_data['取引時点'].astype(str).str[:4].astype(int)

# 築年数を計算（取引年 - 建築年）
test_data['築年数'] = test_data['取引年'] - test_data['建築年']

# 取引月を抽出
test_data['取引月'] = test_data['取引時点'].apply(lambda x: int(str(x)[4:6]))

# 面積/築年数比を計算
test_data['面積/築年数比'] = test_data['面積（㎡）'] / test_data['築年数']

# 駅距離/面積の計算
test_data['駅距離_面積比'] = test_data['最寄駅：距離（分）'] / test_data['面積（㎡）']

# 容積率と建ぺい率の比率
test_data['容積率_建ぺい率比'] = test_data['容積率（％）'] / test_data['建ぺい率（％）']

# 面積と容積率の交互作用
test_data['面積×容積率'] = test_data['面積（㎡）'] * test_data['容積率（％）']

# 建築年と建ぺい率の交互作用
test_data['建築年×建ぺい率'] = test_data['建築年'] * test_data['建ぺい率（％）']

# 特徴量の確認
print(test_data[['取引年', '築年数', '取引月', '面積/築年数比', '駅距離_面積比', '容積率_建ぺい率比', '面積×容積率', '建築年×建ぺい率']].head())

# 新しい特徴量を含むデータを保存
output_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_new_features1.csv'
test_data.to_csv(output_path, index=False)
print(f"新しい特徴量を含むデータを保存しました: {output_path}")


    取引年  築年数  取引月   面積/築年数比   駅距離_面積比  容積率_建ぺい率比   面積×容積率  建築年×建ぺい率
0  2023    7    9  5.000000  0.114286        2.5   7000.0  161280.0
1  2023    7    9  5.714286  0.075000        2.5   8000.0  161280.0
2  2023    3    9  8.333333  0.160000        2.5   5000.0  161600.0
3  2023   10    9  5.000000  0.100000        2.5  10000.0  161040.0
4  2023   23    9  1.521739  0.228571        2.5   7000.0  160000.0
新しい特徴量を含むデータを保存しました: /Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_new_features1.csv


## 目的変数を使用して特徴量を作成する
データリーケージを防ぐためトレインデータはクロスバリデーションを用いて計算しています。

In [3]:
import pandas as pd
import numpy as np

# トレインデータの読み込み
train_data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/train/train_13_27/train_13_27_with_new_features2.csv'
train_data = pd.read_csv(train_data_path, low_memory=False)

# テストデータの読み込み
test_data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_new_features1.csv'
test_data = pd.read_csv(test_data_path, low_memory=False)

# エリア人気度スコアの計算（トレインデータから）
area_popularity = train_data.groupby('市区町村コード')['取引価格（総額）_log'].mean().reset_index()
area_popularity.columns = ['市区町村コード', 'エリア人気度スコア']

# 新しい長期価格トレンドの計算（面積と建築年を使用）
train_data['面積_建築年比'] = train_data['面積（㎡）'] / train_data['建築年']
long_term_trend = train_data.groupby('面積_建築年比')['取引価格（総額）_log'].mean().reset_index()
long_term_trend.columns = ['面積_建築年比', '長期価格トレンド']

# 建築年スコアの計算（トレインデータから）
train_data['建築年'] = train_data['建築年'].astype(int)
year_built_score = train_data.groupby('建築年')['取引価格（総額）_log'].mean().reset_index()
year_built_score.columns = ['建築年', '建築年スコア']

# テストデータにエリア人気度スコアを追加
test_data = test_data.merge(area_popularity, on='市区町村コード', how='left')

# テストデータに長期価格トレンドを追加
test_data['面積_建築年比'] = test_data['面積（㎡）'] / test_data['建築年']
test_data = test_data.merge(long_term_trend, on='面積_建築年比', how='left')

# テストデータに建築年スコアを追加
test_data = test_data.merge(year_built_score, on='建築年', how='left')

# 結果を確認
print(test_data[['エリア人気度スコア', '長期価格トレンド', '建築年スコア']].head())
print(test_data[['エリア人気度スコア', '長期価格トレンド', '建築年スコア']].tail())

# 新しい特徴量を含むデータを保存
output_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_new_features2.csv'
test_data.to_csv(output_path, index=False)
print(f"新しい特徴量を含むデータを保存しました: {output_path}")


   エリア人気度スコア  長期価格トレンド    建築年スコア
0   7.560311  7.581109  7.635302
1   7.560311  7.632346  7.635302
2   7.560311  7.459076  7.655477
3   7.560311  7.664939  7.584905
4   7.560311  7.446411  7.492349
       エリア人気度スコア  長期価格トレンド    建築年スコア
23153   6.759613  7.343557  7.289059
23154   6.759613  7.171763  7.118867
23155   6.759613  7.205725  7.125723
23156   6.759613  7.357532  7.368283
23157   6.759613  7.357532  7.368283
新しい特徴量を含むデータを保存しました: /Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_new_features2.csv


## ワンポットエンコーディングを適用する

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import joblib

# テストデータの読み込み
test_data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_new_features2.csv'
test_data = pd.read_csv(test_data_path, low_memory=False)

# ワンホットエンコーダをロードする
onehot_encoder_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/train/train_13_27/onehot_encoder.pkl'
onehot_encoder = joblib.load(onehot_encoder_path)

# ワンホットエンコーディングを適用するカテゴリ変数
categorical_columns = ['間取り', '建物の構造', '都市計画']

# ワンホットエンコーディングを適用
encoded_categorical_data = onehot_encoder.transform(test_data[categorical_columns])
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=onehot_encoder.get_feature_names_out(categorical_columns))

# 元のデータフレームにエンコードしたデータを結合
test_data = test_data.drop(columns=categorical_columns)
test_data = pd.concat([test_data.reset_index(drop=True), encoded_categorical_df.reset_index(drop=True)], axis=1)

# データの確認
print(test_data.head())

# エンコードされたデータの保存
test_encoded_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_onehot_encoded_features.csv'
test_data.to_csv(test_encoded_path, index=False)
print(f"エンコードされたデータを保存しました: {test_encoded_path}")


         ID        種類  価格情報区分  地域  市区町村コード 都道府県名 市区町村名  地区名  最寄駅：名称  \
0  13000559  中古マンション等  成約価格情報 NaN    13101   東京都  千代田区  岩本町     岩本町   
1  13000560  中古マンション等  成約価格情報 NaN    13101   東京都  千代田区  岩本町     岩本町   
2  13000561  中古マンション等  成約価格情報 NaN    13101   東京都  千代田区  岩本町     岩本町   
3  13000562  中古マンション等  成約価格情報 NaN    13101   東京都  千代田区  岩本町     秋葉原   
4  13000563  中古マンション等  成約価格情報 NaN    13101   東京都  千代田区  岩本町  神田(東京)   

   最寄駅：距離（分）  ...  都市計画_調整区域  都市計画_近隣商業  都市計画_都計外  都市計画_非線引き  都市計画_１中住専  \
0        4.0  ...        0.0        0.0       0.0        0.0        0.0   
1        3.0  ...        0.0        0.0       0.0        0.0        0.0   
2        4.0  ...        0.0        0.0       0.0        0.0        0.0   
3        5.0  ...        0.0        0.0       0.0        0.0        0.0   
4        8.0  ...        0.0        0.0       0.0        0.0        0.0   

  都市計画_１低住専 都市計画_１種住居  都市計画_２中住専  都市計画_２低住専  都市計画_２種住居  
0       0.0       0.0        0.0        0.0        0.0  
1       

## スケーリングを適用する

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

# テストデータの読み込み
data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_with_onehot_encoded_features.csv'
test_data = pd.read_csv(data_path, low_memory=False)

# スケーリングを適用する数値変数のリスト
numeric_columns = [
    '市区町村コード', '最寄駅：距離（分）', '面積（㎡）', '建築年', '建ぺい率（％）', '容積率（％）', 
    '取引時点', '取引年', '築年数', '取引月', '面積/築年数比', '駅距離_面積比', 
    '容積率_建ぺい率比', '面積×容積率', '建築年×建ぺい率', 'エリア人気度スコア', 
    '長期価格トレンド', '建築年スコア'
]

# 数値変数の選択
X_test_numeric = test_data[numeric_columns].copy()

# 無限大や非常に大きな値をチェックして処理
X_test_numeric.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test_numeric.fillna(X_test_numeric.mean(), inplace=True)

# スケーラーを読み込む
scaler_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/train/train_13_27/scaler.pkl'
scaler = joblib.load(scaler_path)

# スケーリングの適用
X_test_numeric_scaled = scaler.transform(X_test_numeric)

# スケーリング後のデータフレームに変換
X_test_numeric_scaled_df = pd.DataFrame(X_test_numeric_scaled, columns=numeric_columns)

# ワンホットエンコーディングされたカテゴリ変数を選択
X_test_categorical = test_data.drop(columns=numeric_columns)

# スケーリングされた数値変数とワンホットエンコーディングされたカテゴリ変数を結合
X_test_scaled_df = pd.concat([X_test_numeric_scaled_df, X_test_categorical.reset_index(drop=True)], axis=1)

# ID列を追加
if 'ID' in test_data.columns:
    X_test_scaled_df['ID'] = test_data['ID'].values

# 前処理済みのデータを保存
test_scaled_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_scaled.csv'
X_test_scaled_df.to_csv(test_scaled_path, index=False)

print(X_test_scaled_df.head())
print(f"予測の準備が出来たデータセットを保存しました: {test_scaled_path}")


    市区町村コード  最寄駅：距離（分）     面積（㎡）       建築年   建ぺい率（％）    容積率（％）      取引時点  \
0 -0.631966  -0.728486 -0.746236  1.431135  1.215581 -0.778029  1.312732   
1 -0.631966  -0.944174 -0.538127  1.431135  1.215581 -0.778029  1.312732   
2 -0.631966  -0.728486 -1.162454  1.740295  1.215581 -0.778029  1.312732   
3 -0.631966  -0.512798 -0.121909  1.199266  1.215581 -0.778029  1.312732   
4 -0.631966   0.134267 -0.746236  0.194497  1.215581 -0.778029  1.312732   

       取引年       築年数       取引月  ...  都市計画_調整区域  都市計画_近隣商業  都市計画_都計外  \
0  1.30881 -0.890499  0.487203  ...        0.0        0.0       0.0   
1  1.30881 -0.890499  0.487203  ...        0.0        0.0       0.0   
2  1.30881 -1.195310  0.487203  ...        0.0        0.0       0.0   
3  1.30881 -0.661891  0.487203  ...        0.0        0.0       0.0   
4  1.30881  0.328743  0.487203  ...        0.0        0.0       0.0   

   都市計画_非線引き  都市計画_１中住専  都市計画_１低住専  都市計画_１種住居  都市計画_２中住専  都市計画_２低住専 都市計画_２種住居  
0        0.0        0.0        0.0  

特徴量のリストを表示する

In [6]:
import pandas as pd
import numpy as np

# テストデータの読み込み
test_data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_scaled.csv'
test_data = pd.read_csv(test_data_path, low_memory=False)

# 列の数を確認
column_count = test_data.shape[1]
print(f"データセットに含まれる列の数: {column_count}")

# カラム名のリストを表示
print(test_data.columns.tolist())


データセットに含まれる列の数: 128
['市区町村コード', '最寄駅：距離（分）', '面積（㎡）', '建築年', '建ぺい率（％）', '容積率（％）', '取引時点', '取引年', '築年数', '取引月', '面積/築年数比', '駅距離_面積比', '容積率_建ぺい率比', '面積×容積率', '建築年×建ぺい率', 'エリア人気度スコア', '長期価格トレンド', '建築年スコア', 'ID', '種類', '価格情報区分', '地域', '都道府県名', '市区町村名', '地区名', '最寄駅：名称', '土地の形状', '間口', '延床面積（㎡）', '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '改装', '取引の事情等', '1953年以前', '面積_建築年比', '間取り_オープンフロア', '間取り_スタジオ', '間取り_メゾネット', '間取り_１ＤＫ', '間取り_１ＤＫ＋Ｓ', '間取り_１Ｋ', '間取り_１Ｋ＋Ｓ', '間取り_１Ｌ', '間取り_１Ｌ＋Ｓ', '間取り_１ＬＤ＋Ｓ', '間取り_１ＬＤＫ', '間取り_１ＬＤＫ＋Ｋ', '間取り_１ＬＤＫ＋Ｓ', '間取り_１ＬＫ', '間取り_１ＬＫ＋Ｓ', '間取り_１Ｒ', '間取り_１Ｒ＋Ｓ', '間取り_２Ｄ', '間取り_２ＤＫ', '間取り_２ＤＫ＋Ｓ', '間取り_２Ｋ', '間取り_２Ｋ＋Ｓ', '間取り_２Ｌ', '間取り_２Ｌ＋Ｓ', '間取り_２ＬＤ', '間取り_２ＬＤ＋Ｓ', '間取り_２ＬＤＫ', '間取り_２ＬＤＫ＋Ｓ', '間取り_２ＬＫ', '間取り_２ＬＫ＋Ｓ', '間取り_３ＤＫ', '間取り_３ＤＫ＋Ｓ', '間取り_３Ｋ', '間取り_３Ｋ＋Ｓ', '間取り_３ＬＤ', '間取り_３ＬＤ＋Ｓ', '間取り_３ＬＤＫ', '間取り_３ＬＤＫ＋Ｋ', '間取り_３ＬＤＫ＋Ｓ', '間取り_３ＬＫ', '間取り_３ＬＫ＋Ｓ', '間取り_４ＤＫ', '間取り_４ＤＫ＋Ｓ', '間取り_４Ｋ', '間取り_４Ｌ', '間取り_４Ｌ＋Ｋ', '間取り_４ＬＤＫ', '間取り_４ＬＤＫ＋Ｋ', '間取り_４ＬＤＫ＋Ｓ', '間取り_５ＤＫ', '間取り_５Ｋ', '間取り_５Ｌ

## 取引価格_logを予測する

In [4]:
import pandas as pd
import numpy as np
import joblib
import os

# モデルとテストデータのパス
model_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/model/ensemble_model_0802_2.joblib'
test_data_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/test/test_13_27/test_13_27_scaled.csv'
submission_path = '/Users/hayakawakazue/Downloads/second_apt_2024_summer/submission/submission_13_27.csv'

# モデルのロード
model = joblib.load(model_path)

# テストデータの読み込み
test_data = pd.read_csv(test_data_path)

# テストデータからID列を抽出
test_ids = test_data['ID']

# モデルのトレーニングに使用した特徴量（順序を一致させる）
selected_features = [
    '長期価格トレンド', 'エリア人気度スコア', '市区町村コード', '面積（㎡）',
    '取引時点', '駅距離_面積比', '面積/築年数比', '建築年スコア', '築年数', 
    '建築年', '建築年×建ぺい率'
]

# 予測に使用する特徴量を選択し、トレーニング時の順序に合わせる
test_features = test_data[selected_features].copy()

# 無限大や非常に大きな値をチェックして処理
test_features.replace([np.inf, -np.inf], np.nan, inplace=True)
test_features.fillna(test_features.mean(), inplace=True)

# 取引価格_logの予測
predictions = model.predict(test_features)

# 予測結果をテストデータに追加
test_data['取引価格（総額）_log'] = predictions

# 必要な列だけを残して他の列を削除
submission_df = test_data[['ID', '取引価格（総額）_log']]

# ディレクトリが存在するか確認し、存在しない場合は作成
save_dir = os.path.dirname(submission_path)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# 提出ファイルを保存
submission_df.to_csv(submission_path, index=False)

print(f"予測結果を保存しました: {submission_path}")


予測結果を保存しました: /Users/hayakawakazue/Downloads/second_apt_2024_summer/submission/submission_13_27.csv
